# Import Packages

In [1]:
import tensorflow as tf
import tensorflow.keras as keras

print(tf.__version__)
print(tf.keras.__version__)

1.13.1
2.2.4-tf


In [2]:
from keras.applications.vgg19 import VGG19
from keras.models import Model,Sequential
from keras.layers import Input, LSTM, Multiply, Dense, Embedding, Flatten, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import RMSprop, Adam


Using TensorFlow backend.


In [3]:
import json
import pandas as pd
import numpy as np
import glob
import os
import spacy
import h5py
import pickle

In [7]:
%cd /home/anthony/Projects/42028-assignment3/
#%cd /content/gdrive/My Drive/42028-DL-CNN/Assignment3/
%ls

/home/anthony/Projects/42028-assignment3
Annotations_Train_mscoco.zip  Pipfile.lock                train2014.zip
Annotations_Val_mscoco.zip    predict/                    train_set_index.csv
app.py                        __pycache__/                uploads/
data/                         Questions_Train_mscoco.zip  val2014.zip
models/                       Questions_Val_mscoco.zip    val_set_index.csv
model_training/               README.md                   VQA/
notebooks/                    settings.py
Pipfile                       templates/


# Prepare Training and Validation Dataframes

In [4]:
def list_images_from_dir(path):
  #img_list = glob.glob(path + '/*.jpg')
  img_list = os.listdir(path)
  img_list = [img.replace(path, '') for img in img_list]
  print(len(img_list))
  return img_list

def remove_missing_images(df, col_name, img_list):
  return df[df[col_name].isin(img_list)]

## Prepare Val

In [8]:
with open('data/coco/annotations/instances_val2014.json') as json_file:  
#with open('data/coco/annotations/annotations/instances_val2014.json') as json_file:  
    data = json.load(json_file)

In [9]:
images_df = pd.DataFrame(data['images'])
annotations_df = pd.DataFrame(data['annotations'])
categories_df = pd.DataFrame(data['categories'])
images_df = images_df[['file_name','height','id','width']]
annotations_df = annotations_df[['category_id','image_id']]
categories_df = categories_df[['id', 'name']]

val_set = annotations_df.merge(images_df, how='left', left_on='image_id', right_on='id').drop('id', axis=1)
val_set = val_set.merge(categories_df, how='left', left_on='category_id', right_on='id').drop('id', axis=1)

In [10]:
val_set.head()

,category_id,image_id,file_name,height,width,name
0,58,558840,COCO_val2014_000000558840.jpg,427,640,hot dog
1,58,200365,COCO_val2014_000000200365.jpg,480,640,hot dog
2,58,200365,COCO_val2014_000000200365.jpg,480,640,hot dog
3,58,200365,COCO_val2014_000000200365.jpg,480,640,hot dog
4,58,200365,COCO_val2014_000000200365.jpg,480,640,hot dog


In [12]:
val_dir = "data/coco/images/val2014/"
val_img_list_file = 'data/coco/images/val2014_img_list.pkl'

In [13]:
#val_images = list_images_from_dir(val_dir)
#with open(val_img_list_file, 'wb') as fp:
#    pickle.dump(val_images, fp)

In [14]:
with open (val_img_list_file, 'rb') as fp:
    val_images = pickle.load(fp)
len(val_images)

40504

In [15]:
val_set = remove_missing_images(val_set, 'file_name', val_images)

val_set.shape, val_set['name'].nunique()

((291875, 6), 80)

# Train Image Model

In [0]:
%mkdir models
%mkdir models/vgg19

mkdir: cannot create directory ‘models’: File exists
mkdir: cannot create directory ‘models/vgg19’: File exists


In [16]:
test_datagen = ImageDataGenerator(rescale=1./255)

validation_generator=test_datagen.flow_from_dataframe(
    dataframe=val_set, 
    directory=val_dir, 
    x_col="file_name", 
    y_col="name", 
    class_mode="categorical", 
    target_size=(150, 150), 
    batch_size=32)

Found 40137 images belonging to 80 classes.


# Update model to extract Feature Maps Layer

In [17]:
model_name = f'models/vgg19/vgg19_feature_extractor.h5'

In [18]:
model_feat_extract = keras.models.load_model(model_name)

Instructions for updating:
Colocations handled automatically by placer.


# Extract Feature Maps

In [ ]:
step_size_val=validation_generator.n//32
val_preds = model_feat_extract.predict_generator(validation_generator, max_queue_size=32, workers=-1, use_multiprocessing=True, verbose=1, steps=step_size_val)

with h5py.File('vgg19_img_features_val.h5','w') as hf:
  hf.create_dataset('val', data=val_preds)

In [0]:
with h5py.File(img_feature_file,'r') as hf:
  train_img_feats =  np.array(hf.get('train'))
  val_img_feats =  np.array(hf.get('val'))